# Data Mining Project

### Imports

In [162]:
from numpy import genfromtxt
import pandas as pd
import numpy as np
import operator, re, string, codecs, nltk
from statistics import mean
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
nltk.download('punkt')
from xml.dom import minidom
from string import punctuation
from enum import Enum
try:
    maketrans = ''.maketrans
except AttributeError:
    from string import maketrans
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
from collections import Counter

# Test

# from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
# from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
# from sklearn import decomposition, ensemble

# import xgboost, textblob
# from keras.preprocessing import text, sequence
# from keras import layers, models, optimizers

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\chris\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Program base

In [181]:

class Type(Enum):
    REAL_CSV_SETS = 1
    TEST_ON_TRAINING_SET = 2

class ArticlePredictorBase:
    def __init__(self,test_type,path_to_csv_train,path_to_csv_test):
        if Type.REAL_CSV_SETS == test_type:
            self.type = test_type
            
            #Import of the test set
            try:
                test_csv_file = path_to_csv_test
                test = pd.read_csv(test_csv_file,delimiter=',')
                #TODO self.valid_x, self.valid_y in this case
                print("Not yet implemented")
            except:
                print("Error while importing testing set csv file") 
                self.type = None
            
            #Import of the train set
            try:
                train_csv_file = path_to_csv_train
                train = pd.read_csv(train_csv_file,delimiter=',')
                #TODO self.train_x, self.train_y in this case
                print("Not yet implemented")
            except:
                print("Error while importing training set csv file") 
                self.type = None
                
        elif Type.TEST_ON_TRAINING_SET == test_type:
            self.type = test_type
            
            #Import of the train/testing set
            try:
                train_csv_file = path_to_csv_train
                data = pd.read_csv(train_csv_file,delimiter=',')
            except:
                print("Error while importing the csv file") 
                self.type = None
                
            #X = pd.get_dummies(data.iloc[:,3]) # On met quoi ici??
            y = data.iloc[:,2]
            
            print(y)
            
            #print(pd.get_dummies(data.iloc[:,:3])) #test
            #self.train_x, self.valid_x, self.train_y, self.valid_y = model_selection.train_test_split(theSet['text'], theSet['label'])
        else:
            self.type = None
            print('Error type')

    def test(self):
        print("(Testing Launched)")
        if self.type is None:
            print("Initialisation error ")
            return None
        
        test_x = self.test_x
        test_y = self.test_y
            
    def train(self):
        print("(Training Launched)")
        if self.type is None:
            print("Initialisation error ")
            return None
            
        train_x = self.train_x
        train_y = self.train_y
         
        # for i in range(0,train.size//3):
        #    file_path = train['file'][i]
        #    excepted = train['earnings: 0 no/ 1 yes'][i]
        #    print(file_path)
            #xmldoc = minidom.parse('project/project/'+file_path)
    
# predictor = ArticlePredictorBase(Type.REAL_CSV_SETS,"project/project/train.csv","project/project/test.csv")
predictor = ArticlePredictorBase(Type.TEST_ON_TRAINING_SET,"project/project/train.csv","project/project/test.csv")
# predictor.train()

0       0
1       0
2       0
3       0
4       0
5       0
6       0
7       0
8       1
9       0
10      1
11      1
12      1
13      1
14      0
15      0
16      0
17      1
18      0
19      0
20      0
21      1
22      1
23      0
24      0
25      1
26      0
27      0
28      0
29      0
       ..
4770    0
4771    1
4772    0
4773    1
4774    0
4775    0
4776    0
4777    0
4778    0
4779    0
4780    0
4781    0
4782    0
4783    0
4784    0
4785    0
4786    0
4787    0
4788    0
4789    1
4790    1
4791    0
4792    0
4793    0
4794    0
4795    0
4796    0
4797    1
4798    1
4799    0
Name: earnings: 0 no/ 1 yes, Length: 4800, dtype: int64


## Computation of 1 file

We test here with 1.xml data file

### Importation of the file

In [165]:
xmldoc = minidom.parse('project/project/data/1.xml')
itemlist = xmldoc.getElementsByTagName('BODY')

#Verification

if len(itemlist) != 1:
    print('Error: XML file invalid')
    sys.exit(0)

text = itemlist[0].childNodes[0].nodeValue

### Preprocessing

In [166]:
#Preprocess
text = text.lower()    
text = re.sub(r'\d+', '', text) #Deleting numbers
text = text.translate(str.maketrans('','',string.punctuation)) #Deleting ponctuation

#Preprocess ideas : https://medium.com/@datamonsters/text-preprocessing-in-python-steps-tools-and-examples-bf025f872908
     
#Tokenization    
tokens = word_tokenize(text)
print(str(len(tokens)) + " tokens BEFORE stop words removing")
tokens = [i for i in tokens if not i in ENGLISH_STOP_WORDS]
print(str(len(tokens)) + " tokens AFTER stop words removing")

#print(tokens)

#stemmer= PorterStemmer()
#tokens = set(map(stemmer.stem,tokens))
#print (tokens)

438 tokens BEFORE stop words removing
248 tokens AFTER stop words removing


In [167]:
def most_common_terms(terms):
    terms_count_array = []
    for term in terms:
        terms_count_array += term.split(" ")
    counter = Counter(terms_count_array)
    for word in counter.most_common():
        print(word)
        
most_common_terms(tokens)

('dlrs', 14)
('new', 9)
('york', 8)
('sales', 7)
('times', 7)
('cocoa', 6)
('comissaria', 5)
('smith', 5)
('said', 5)
('bags', 5)
('mln', 5)
('crop', 5)
('bahia', 4)
('february', 3)
('total', 3)
('ports', 3)
('junejuly', 3)
('augsept', 3)
('marchapril', 3)
('octdec', 3)
('dec', 3)
('week', 2)
('temporao', 2)
('period', 2)
('year', 2)
('arrivals', 2)
('kilos', 2)
('consignment', 2)
('figures', 2)
('farmers', 2)
('shippers', 2)
('sold', 2)
('bean', 2)
('shipment', 2)
('limited', 2)
('tonne', 2)
('open', 2)
('july', 2)
('butter', 2)
('sept', 2)
('currency', 2)
('areas', 2)
('uruguay', 2)
('showers', 1)
('continued', 1)
('zone', 1)
('alleviating', 1)
('drought', 1)
('early', 1)
('january', 1)
('improving', 1)
('prospects', 1)
('coming', 1)
('normal', 1)
('humidity', 1)
('levels', 1)
('restored', 1)
('weekly', 1)
('review', 1)
('dry', 1)
('means', 1)
('late', 1)
('ended', 1)
('making', 1)
('cumulative', 1)
('season', 1)
('stage', 1)
('delivered', 1)
('earlier', 1)
('included', 1)
('doubt', 